# Summarizing uncertainty in scaling variables

Leanne Friedrich 3/27/24

## initialization

In [1]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [2]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.append("../")
sys.path.append("../../")
from py.tools.config import cfg
import py.tools.logs as logs
from plainIm import plainIm
import matplotlib.pyplot as plt
import py.metrics.m_plot.m_plots as mp
import py.metrics.m_SDT as me
import regression as rg
import matplotlib.cm as cm
import py.file.file_handling as fh
LOGGERDEFINED = logs.openLog('uncertainty.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

## analyzing data

In [3]:
# get an object that holds summary data for all files
ms = me.summarySDT(os.path.join(cfg.path.fig, 'SDT', 'summaries', 'XS+zSDTSummaries.csv')
                             , diag=True)

Independents: 
	 meta        :  bn, calibFile, fluFile, printFolderR
	                date
	 meta sup    :  sup_base, sup_days, sup_dye, sup_rheModifier, sup_shortname, sup_surfactant, sup_surfactantWt, sup_type, sup_var
	 meta ink    :  ink_base, ink_days, ink_dye, ink_rheModifier, ink_shortname, ink_surfactant, ink_surfactantWt, ink_type, ink_var
	 const       :  GaRatio, GdRatio, gname, GtaRatio, int_Ca, int_CaInv, int_Re, ReRatio, tau0aRatio, tau0dRatio, tGdRatio, viscRatio, vRatio
	                dEst, hDragP, pressureCh0, rGrava, rGravd, sigma, spacing, spacing_adj, vDragP, zdepth
	 const sup   :  sup_Bma, sup_Bmd, sup_Ca, sup_CaInv, sup_diluent, sup_diluentWt, sup_dnorma, sup_dnorma_adj, sup_dnormd, sup_dnormd_adj, sup_dnormInva, sup_dnormInvd, sup_Oh, sup_OhInv, sup_Re, sup_We, sup_WeInv
	                sup_density, sup_dPRa, sup_dPRd, sup_eta0a, sup_eta0d, sup_Gstora, sup_Gstord, sup_rate, sup_tau0a, sup_tau0d, sup_v, sup_val, sup_visc0
	 const ink   :  ink_Bma, ink_Bmd, ink

,1,2,3
wp,X_w1p,X_w2p,X_w3p
wo,X_w1o,X_w2o,X_w3o
wrelax,delta_X_w1relax,delta_X_w2relax,delta_X_w3relax
write,delta_X_write1,delta_X_write2,
dp,X_d1p,X_d2p,
do,X_d1o,X_d2o,
drelax,delta_X_d1relax,delta_X_d2relax,
disturb,delta_X_disturb1,delta_X_disturb2,


In [15]:
out = []
for i,row in ms.ss.iterrows():
    d = {}
    d['derr'] = 0.001  # mm
    d['dierrfrac'] = d['derr']/0.6
    d['dserrfrac'] = d['derr']/0.9
    d['desterr'] = 0.01 # mm
    d['desterrfrac'] = d['desterr']/row['dEst']
    d['dacterr'] = 0.014  # mm
    d['dacterrfrac'] = d['dacterr']/row['dEst']
    d['Derr'] = 0.0064 # mm
    d['verr'] = 0.01 # mm/s
    d['sigma'] = row['sigma']  # mN/m
    d['sigmaerr'] = {33.6558:0.228794, 7.905:0.107424, 37.8013:0.03415, 1.7649:0.003311, 15.43863469:2.970641}[d['sigma']]  # mN/m
    d['sigmaerrfrac'] = d['sigmaerr']/d['sigma']
    d['spacingerr'] = np.sqrt(d['derr']**2+d['desterr']**2) 
    d['spacingadjerr'] = np.sqrt(d['derr']**2+d['dacterr']**2)
    d['rhoerr'] = 0.1 # g/mL
    d['rhoerrfrac'] = d['rhoerr']/row['ink_density']
    for fluid in ['i', 's']:
        
        ful = {'i':'ink', 's':'sup'}[fluid]
        d[f'v{fluid}errfrac'] = d['verr']/row[f'{ful}_v']
        d[f'eta{fluid}'] = row[f'{ful}_eta0d']
        d[f'eta{fluid}errfrac'] = d[f'eta{fluid}']*0.03
        d[f'Ca{fluid}errfrac'] = np.sqrt(0.03**2 + d[f'v{fluid}errfrac']**2+d['sigmaerrfrac']**2)
        d[f'Oh{fluid}errfrac'] = np.sqrt(0.03**2 + 1/2*d['rhoerrfrac']**2 + 1/2*d[f'd{fluid}errfrac']**2) 
        d[f'Re{fluid}errfrac'] = np.sqrt(d['rhoerrfrac']**2 + d[f'v{fluid}errfrac']**2+ d[f'd{fluid}errfrac']**2+0.03**2)
        d[f'We{fluid}errfrac'] = np.sqrt(d['rhoerrfrac']**2 + d[f'v{fluid}errfrac']**2+ d['sigmaerrfrac']**2+0.03**2)
        for dire in ['a', 'd']:
            d[f'tauy{fluid}{dire}'] = row[f'{ful}_tau0{dire}']
            d[f'tauy{fluid}{dire}errfrac'] = d[f'tauy{fluid}{dire}']*0.08
            d[f'Bm{fluid}{dire}errfrac'] = np.sqrt(0.08**2+d[f'd{fluid}errfrac']**2+0.03**2+d[f'v{fluid}errfrac']**2)
            for dd in ['e', 'a']:
                der = {'e':'desterrfrac', 'a':'dacterrfrac'}[dd]
                d[f'DR{fluid}{dire}{dd}errfrac'] = np.sqrt(d[der]**2 + 0.08**2 +  d['sigmaerrfrac']**2)
    out.append(d)
df1 = pd.DataFrame(out)

In [23]:
df1.to_csv(os.path.join(cfg.path.fig, 'SDT', 'summaries', 'errors.csv'))

In [21]:
def minMax(x):
    return pd.Series(index=['min','max'],data=[x.min()*100,x.max()*100])


df2 = df1.apply(minMax).transpose()
df2.sort_index()

,min,max
Bmiaerrfrac,8.55,8.55
Bmiderrfrac,8.55,8.55
Bmsaerrfrac,8.55,8.55
Bmsderrfrac,8.55,8.55
Caierrfrac,3.00,19.47
Caserrfrac,3.00,19.47
DRiaaerrfrac,8.33,20.97
DRiaeerrfrac,8.17,20.90
DRidaerrfrac,8.33,20.97
DRideerrfrac,8.17,20.90


In [157]:
min(ms.ss.ink_Bma.min(), ms.ss.sup_Bma.min(), ms.ss.ink_Bmd.min(), ms.ss.sup_Bmd.min()), max(ms.ss.ink_Bma.max(), ms.ss.sup_Bma.max(), ms.ss.ink_Bmd.max(), ms.ss.sup_Bmd.max())

(0.0030177344246356, 0.8865363838257969)

In [158]:
min(ms.ss.ink_Re.min(), ms.ss.sup_Re.min()), max(ms.ss.ink_Re.max(), ms.ss.sup_Re.max())

(0.0003365556726152, 0.032943054363007)

In [159]:
min(ms.ss.ink_Ca.min(), ms.ss.sup_Ca.min(), ms.ss.int_Ca.min()), max(ms.ss.ink_Ca.max(), ms.ss.sup_Ca.max(), ms.ss.int_Ca.max())

(0.23979286219008333, 52.383936645657904)

In [160]:
min(ms.ss.ink_dnorma.min(), ms.ss.sup_dnorma.min(), ms.ss.ink_dnormd.min(), ms.ss.sup_dnormd.min()), max(ms.ss.ink_dnorma.max(), ms.ss.sup_dnorma.max(), ms.ss.ink_dnormd.max(), ms.ss.sup_dnormd.max())

(0.0007236311750129249, 17.695533219761497)

In [161]:
min(ms.ss.ink_dnorma_adj.min(), ms.ss.sup_dnorma_adj.min(), ms.ss.ink_dnormd_adj.min(), ms.ss.sup_dnormd_adj.min()), max(ms.ss.ink_dnorma_adj.max(), ms.ss.sup_dnorma_adj.max(), ms.ss.ink_dnormd_adj.max(), ms.ss.sup_dnormd_adj.max())

(0.00043439586141566457, 14.26243620229895)

In [162]:
min(ms.ss.ink_Oh.min(), ms.ss.sup_Oh.min()), max(ms.ss.ink_Oh.max(), ms.ss.sup_Oh.max())

(5.665099662560709, 250.35986233867428)

In [163]:
min(ms.ss.ink_We.min(), ms.ss.sup_We.min()), max(ms.ss.ink_We.max(), ms.ss.sup_We.max())

(0.0017916674094806836, 0.22659666479443388)